In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd
from sklearn import preprocessing


proximity_u = './data/0_Golden_Standard/proximity_clean_05.csv'
capital_d = './data/0_Golden_Standard/capdist_km.csv'
cow2iso = './data/0_Golden_Standard/cow2iso.csv'

In [2]:
prox = pd.read_csv(proximity_u,sep=',')
cap = pd.read_csv(capital_d,sep=',')
cow2iso = pd.read_csv(cow2iso,sep=',')

In [3]:
prox = prox.rename(columns={"iso_code": "iso_code_src"})
prox['iso_code_trg'] = prox['trg']
prox

,Unnamed: 0,src,trg,iso_code_src,region,iso_code_trg
0,0,Afghanistan,China,AFG,Asia,China
1,1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,Iran (Islamic Republic of)
2,2,Afghanistan,Pakistan,AFG,Asia,Pakistan
3,3,Afghanistan,Tajikistan,AFG,Asia,Tajikistan
4,4,Afghanistan,Turkmenistan,AFG,Asia,Turkmenistan
...,...,...,...,...,...,...
717,791,Zambia,Zimbabwe,ZMB,Africa,Zimbabwe
718,792,Zimbabwe,Botswana,ZWE,Africa,Botswana
719,793,Zimbabwe,Mozambique,ZWE,Africa,Mozambique
720,794,Zimbabwe,South Africa,ZWE,Africa,South Africa


In [4]:
prox.loc[prox['iso_code_src'] == 'MDA']

,Unnamed: 0,src,trg,iso_code_src,region,iso_code_trg
431,465,"Moldova, Republic of",Romania,MDA,Europe,Romania
432,466,"Moldova, Republic of",Ukraine,MDA,Europe,Ukraine


In [5]:
#checking Nan's from prox
nans_check = prox[prox.isna().any(axis=1)]
nans_check

,Unnamed: 0,src,trg,iso_code_src,region,iso_code_trg


In [6]:
cap['cow_code_src'] = cap['src']
cap['cow_code_trg'] = cap['trg']
cap

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg
0,USA,CAN,731,0.001367989056,USA,CAN
1,USA,BHM,1623,0.0006161429452,USA,BHM
2,USA,CUB,1813,0.0005515719801,USA,CUB
3,USA,HAI,2286,0.0004374453193,USA,HAI
4,USA,DOM,2358,0.0004240882103,USA,DOM
...,...,...,...,...,...,...
41001,WSM,SOL,1581,0.0006325110689,WSM,SOL
41002,WSM,FJI,823,0.001215066829,WSM,FJI
41003,WSM,MSI,2555,0.0003913894325,WSM,MSI
41004,WSM,PAL,4786,0.0002089427497,WSM,PAL


In [7]:
#no longer active countries / changed name
#consider erasing

confl_l= ["AUH",
       "GDR",
       "CZR",
       "YUG",
       "DRC",
       "SAF",
       "TRA",
       "YAR",
       "ROK",
       "ZAN",
       "OFS",
       "SYR",
       "YPR",
       "TBT",
       "RVN",
       "MCO",
       "MNC",
       "LIE",
       "AND",
       "SMR",
       "SNM",
       "DRV"]


#verify conflicts not in proximity_u
#confl_rows = prox[prox.src.str.contains('|'.join(confl_l))]
#confl_rows += prox[prox.trg.str.contains('|'.join(confl_l))]
#confl_rows

In [8]:
# Cheeky trick to initialize empty pds index
#indexNames = df[ df['src'] == "POTATOE" ].index

def confl_c(df,l):
    for string in l:
        if df['src'].str.contains(string).any():
            df = df[~df.src.str.contains(string)]
        if df['trg'].str.contains(string).any():
            df = df[~df.trg.str.contains(string)]
        if ~df['src'].str.contains(string).any() and ~df['trg'].str.contains(string).any():
            print("{} has been removed from the dataset".format(string))
    return df#



In [9]:
# Delete these row indexes from dataFrame
cap = confl_c(cap,confl_l)

AUH has been removed from the dataset
GDR has been removed from the dataset
CZR has been removed from the dataset
YUG has been removed from the dataset
DRC has been removed from the dataset
SAF has been removed from the dataset
TRA has been removed from the dataset
YAR has been removed from the dataset
ROK has been removed from the dataset
ZAN has been removed from the dataset
OFS has been removed from the dataset
SYR has been removed from the dataset
YPR has been removed from the dataset
TBT has been removed from the dataset
RVN has been removed from the dataset
MCO has been removed from the dataset
MNC has been removed from the dataset
LIE has been removed from the dataset
AND has been removed from the dataset
SMR has been removed from the dataset
SNM has been removed from the dataset
DRV has been removed from the dataset


In [10]:
#verifying there are no 0 distances
cap.loc[cap['kmdist'] == 0]

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg


In [11]:
# Creating a dict of ISOs [keys] and src [values]

def create_dicts_from_df(df, col1, col2):
    d = {}
    i_d = {}
    for index, row in df.iterrows():
        if row[col1] in d:
            #key already in dict 1"
            pass
        
        if row[col2] in i_d:
            #key already in dict 2"
            pass
        
        else:
            d[row[col1]] = row[col2]
            i_d[row[col2]] = row[col1]
    
    return d, i_d

di_src, swapped_d = create_dicts_from_df(prox, "iso_code_src", "src")

In [12]:
di_src

{'AFG': 'Afghanistan',
 'ALB': 'Albania',
 'DZA': 'Algeria',
 'ASM': 'American Samoa',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ATG': 'Antigua and Barbuda',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ABW': 'Aruba',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BHS': 'Bahamas',
 'BHR': 'Bahrain',
 'BGD': 'Bangladesh',
 'BRB': 'Barbados',
 'BLR': 'Belarus',
 'BEL': 'Belgium',
 'BLZ': 'Belize',
 'BEN': 'Benin',
 'BMU': 'Bermuda',
 'BTN': 'Bhutan',
 'BOL': 'Bolivia',
 'BIH': 'Bosnia and Herzegovina',
 'BWA': 'Botswana',
 'BRA': 'Brazil',
 'BRN': 'Brunei',
 'BGR': 'Bulgaria',
 'BFA': 'Burkina Faso',
 'BDI': 'Burundi',
 'CPV': 'Cabo Verde',
 'KHM': 'Cambodia',
 'CMR': 'Cameroon',
 'CAN': 'Canada',
 'CYM': 'Cayman Islands',
 'CAF': 'Central African Republic',
 'TCD': 'Chad',
 'CHL': 'Chile',
 'CHN': 'China',
 'CXR': 'Christmas Island',
 'CCK': 'Cocos (Keeling) Islands',
 'COL': 'Colombia',
 'COM': 'Comoros',
 'COG': 'Congo',
 'COD': 'Congo, Democratic Republic of the',
 'COK':

In [13]:
swapped_d

{'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Algeria': 'DZA',
 'American Samoa': 'ASM',
 'Angola': 'AGO',
 'Anguilla': 'AIA',
 'Antigua and Barbuda': 'ATG',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Aruba': 'ABW',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Bahamas': 'BHS',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Barbados': 'BRB',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BLZ',
 'Benin': 'BEN',
 'Bermuda': 'BMU',
 'Bhutan': 'BTN',
 'Bolivia': 'BOL',
 'Bosnia and Herzegovina': 'BIH',
 'Botswana': 'BWA',
 'Brazil': 'BRA',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Burkina Faso': 'BFA',
 'Burundi': 'BDI',
 'Cabo Verde': 'CPV',
 'Cambodia': 'KHM',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Cayman Islands': 'CYM',
 'Central African Republic': 'CAF',
 'Chad': 'TCD',
 'Chile': 'CHL',
 'China': 'CHN',
 'Christmas Island': 'CXR',
 'Cocos (Keeling) Islands': 'CCK',
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Congo': 'COG',
 'Congo, Democratic Republic of the': 'COD',
 'Cook 

In [14]:
# Manually adding NaN ISO Codes
di_src["WSM"] = "Samoa"
di_src["KNA"] = "Saint Kitts and Nevis"

swapped_d["Samoa"] = "WSM"
swapped_d["Saint Kitts and Nevis"] = "WSM"

#Adding more countries added to db into to the dictionary
di_src.update({"CZE": 'Czechia'})
swapped_d.update({'Czechia':"CZE"})

di_src.update({"TZA": "Tanzania, United Republic of"})
swapped_d.update({"Tanzania, United Republic of": "TZA"})

di_src.update({"MDA": "Moldova, Republic of"})
swapped_d.update({"Moldova, Republic of": "MDA"})

di_src.update({"KOR": "Korea, Republic of"})
swapped_d.update({"Korea, Republic of": "KOR"})

In [15]:
#Filling ISO_CODES_TRG from trg and a dict


def replace_df_values_w_dic(df, col1, col2, i_d):
    #new_df = df.copy(deep=True)
    nan_dict = {}    
    for index, row in df.iterrows():

        if row[col1] in i_d:
            #at method avoids running into problems when modifying the df and iterrows https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index
            df.at[index,col2] = i_d[row[col1]]
        else:
            #print("NaN Alert with {}".format(df[col2][index]))
            if df[col2][index] in nan_dict:
                pass
            else:
                nan_dict[df[col2][index]] = "?"
            df.at[index,col2] = np.nan
    if nan_dict != {}:
        print("NaN Alert")
    return df, nan_dict

#careful as the value call is inverse in this function
prox_u, nan_d = replace_df_values_w_dic(prox, "trg", "iso_code_trg", swapped_d)

In [16]:
if 'Unnamed: 0' in prox_u.columns:
    prox_u = prox_u.drop(["Unnamed: 0"],axis=1)
prox_u

,src,trg,iso_code_src,region,iso_code_trg
0,Afghanistan,China,AFG,Asia,CHN
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,IRN
2,Afghanistan,Pakistan,AFG,Asia,PAK
3,Afghanistan,Tajikistan,AFG,Asia,TJK
4,Afghanistan,Turkmenistan,AFG,Asia,TKM
...,...,...,...,...,...
717,Zambia,Zimbabwe,ZMB,Africa,ZWE
718,Zimbabwe,Botswana,ZWE,Africa,BWA
719,Zimbabwe,Mozambique,ZWE,Africa,MOZ
720,Zimbabwe,South Africa,ZWE,Africa,ZAF


In [17]:
cap

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg
0,USA,CAN,731,0.001367989056,USA,CAN
1,USA,BHM,1623,0.0006161429452,USA,BHM
2,USA,CUB,1813,0.0005515719801,USA,CUB
3,USA,HAI,2286,0.0004374453193,USA,HAI
4,USA,DOM,2358,0.0004240882103,USA,DOM
...,...,...,...,...,...,...
41001,WSM,SOL,1581,0.0006325110689,WSM,SOL
41002,WSM,FJI,823,0.001215066829,WSM,FJI
41003,WSM,MSI,2555,0.0003913894325,WSM,MSI
41004,WSM,PAL,4786,0.0002089427497,WSM,PAL


In [18]:
cow2iso

,cow_id,cow3,iso_id,iso2,iso3,valid_from,valid_until,cname,cname_full,comments,statenme
0,2.0,USA,841.0,US,USA,1962.0,1980.0,USA (before 1981),USA and Puerto Rico,Including Puerto Rico,United States of America
1,2.0,USA,842.0,US,USA,1981.0,NaN,USA,"USA, Puerto Rico and US Virgin Islands",Including Puerto Rico and US Virgin Islands,United States of America
2,20.0,CAN,124.0,CA,CAN,1962.0,NaN,Canada,Canada,NaN,Canada
3,31.0,BHM,44.0,BS,BHS,1962.0,NaN,Bahamas,Bahamas,NaN,Bahamas
4,40.0,CUB,192.0,CU,CUB,1962.0,NaN,Cuba,Cuba,NaN,Cuba
...,...,...,...,...,...,...,...,...,...,...,...
310,NaN,NaN,866.0,VD,VDR,1962.0,1974.0,Fmr Dem. Rep. of Vietnam,Former Democratic Republic of Vietnam,NaN,NaN
311,NaN,NaN,92.0,VG,VGB,1962.0,NaN,Br. Virgin Isds,British Virgin Islands,NaN,NaN
312,NaN,NaN,850.0,VI,VIR,1962.0,1980.0,US Virgin Isds,US Virgin Islands,NaN,NaN
313,NaN,NaN,0.0,WL,WLD,1962.0,NaN,World,World,NaN,NaN


In [19]:
cow2iso_src = cow2iso.rename(columns={'cow3':'cow_code_src', 'iso3':'iso_code_src'}).drop(['cow_id', 
                                                                                           'iso_id',
                                                                                           'iso2',
                                                                                           'valid_from',
                                                                                           'valid_until',
                                                                                           'cname',
                                                                                           'cname_full',
                                                                                           'comments',
                                                                                           'statenme'],axis=1)


In [20]:
# changes in cow2iso (mistakes in the ISO Starndard or quick fixes)

#indexes manually found using code one cell below
cow2iso_src['iso_code_src'] = cow2iso_src['iso_code_src'].replace(['ATA'],'ATG')
cow2iso_src.at[45, 'iso_code_src'] = 'MCO'
cow2iso_src.at[46, 'iso_code_src'] = 'LIE'
cow2iso_src.at[66, 'iso_code_src'] = 'CZE'
cow2iso_src.at[187, 'iso_code_src'] = 'TWN'
cow2iso_src.at[188, 'iso_code_src'] = 'KOR'

#cow2iso_src.loc[cow2iso_src['cow_code_src'] == 'MNC']['iso_code_src'] = 'MCO'
cow2iso_src = cow2iso_src.append({"cow_code_src":'UK',"iso_code_src":'GBR'}, ignore_index=True)
cow2iso_src = cow2iso_src.append({"cow_code_src":'SER',"iso_code_src":'SRB'}, ignore_index=True)
cow2iso_src = cow2iso_src.append({"cow_code_src":'RUM',"iso_code_src":'ROU'}, ignore_index=True)
cow2iso_src = cow2iso_src.append({"cow_code_src":'VNM',"iso_code_src":'VNM'}, ignore_index=True)
cow2iso_src = cow2iso_src.append({"cow_code_src":'FJI',"iso_code_src":'FJI'}, ignore_index=True)
cow2iso_src = cow2iso_src.append({"cow_code_src":'TAZ',"iso_code_src":'TZA'}, ignore_index=True)
cow2iso_src = cow2iso_src.append({"cow_code_src":'MLD',"iso_code_src":'MDA'}, ignore_index=True)
cow2iso_src

,cow_code_src,iso_code_src
0,USA,USA
1,USA,USA
2,CAN,CAN
3,BHM,BHS
4,CUB,CUB
...,...,...
317,RUM,ROU
318,VNM,VNM
319,FJI,FJI
320,TAZ,TZA


In [21]:
cow2iso_src.loc[cow2iso_src['cow_code_src'] == 'FJI']

,cow_code_src,iso_code_src
319,FJI,FJI


In [22]:
cow2iso_trg = cow2iso_src.rename(columns={'cow_code_src':'cow_code_trg','iso_code_src':'iso_code_trg'})
cow2iso_trg

,cow_code_trg,iso_code_trg
0,USA,USA
1,USA,USA
2,CAN,CAN
3,BHM,BHS
4,CUB,CUB
...,...,...
317,RUM,ROU
318,VNM,VNM
319,FJI,FJI
320,TAZ,TZA


In [23]:
cap = pd.merge(cap,cow2iso_src,on=["cow_code_src"], how = "left")
cap = pd.merge(cap,cow2iso_trg,on=["cow_code_trg"], how = "left")
cap

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg,iso_code_src,iso_code_trg
0,USA,CAN,731,0.001367989056,USA,CAN,USA,CAN
1,USA,CAN,731,0.001367989056,USA,CAN,USA,CAN
2,USA,BHM,1623,0.0006161429452,USA,BHM,USA,BHS
3,USA,BHM,1623,0.0006161429452,USA,BHM,USA,BHS
4,USA,CUB,1813,0.0005515719801,USA,CUB,USA,CUB
...,...,...,...,...,...,...,...,...
37031,WSM,SOL,1581,0.0006325110689,WSM,SOL,WSM,SLB
37032,WSM,FJI,823,0.001215066829,WSM,FJI,WSM,FJI
37033,WSM,MSI,2555,0.0003913894325,WSM,MSI,WSM,MHL
37034,WSM,PAL,4786,0.0002089427497,WSM,PAL,WSM,PLW


In [24]:
cap.loc[cap['cow_code_src'] == 'DRV'].sort_values(by=['kmdist'])

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg,iso_code_src,iso_code_trg


In [25]:
cap, nan_d1 = replace_df_values_w_dic(cap, "iso_code_trg", "trg", di_src)
cap, nan_d2 = replace_df_values_w_dic(cap, "iso_code_src", "src", di_src)

In [26]:
nan_d1

{}

In [27]:
nan_d2

{}

In [28]:
#checking Nan's from the capital list
cap_nans = cap[cap.isna().any(axis=1)]
cap_nans

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg,iso_code_src,iso_code_trg


In [29]:
cap

,src,trg,kmdist,Proximity_1_over_dist,cow_code_src,cow_code_trg,iso_code_src,iso_code_trg
0,United States of America,Canada,731,0.001367989056,USA,CAN,USA,CAN
1,United States of America,Canada,731,0.001367989056,USA,CAN,USA,CAN
2,United States of America,Bahamas,1623,0.0006161429452,USA,BHM,USA,BHS
3,United States of America,Bahamas,1623,0.0006161429452,USA,BHM,USA,BHS
4,United States of America,Cuba,1813,0.0005515719801,USA,CUB,USA,CUB
...,...,...,...,...,...,...,...,...
37031,Samoa,Solomon Islands,1581,0.0006325110689,WSM,SOL,WSM,SLB
37032,Samoa,Fiji,823,0.001215066829,WSM,FJI,WSM,FJI
37033,Samoa,Marshall Islands,2555,0.0003913894325,WSM,MSI,WSM,MHL
37034,Samoa,Palau,4786,0.0002089427497,WSM,PAL,WSM,PLW


In [30]:
#dropping columns from cap to facilitate the merge
cap_clean = cap.drop(columns=['src','trg','cow_code_src', 'cow_code_trg'],axis=1)

In [31]:
#checking Nan's from prox_u before merge
nans_before = prox_u[prox_u.isna().any(axis=1)]
nans_before

,src,trg,iso_code_src,region,iso_code_trg


In [32]:
# hopefully ready to merge
prox_final = pd.merge(prox_u,cap_clean,on=["iso_code_src", "iso_code_trg"], how = "left")
prox_final

,src,trg,iso_code_src,region,iso_code_trg,kmdist,Proximity_1_over_dist
0,Afghanistan,China,AFG,Asia,CHN,4199.0,0.0002381519409
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,IRN,1618.0,0.0006180469716
2,Afghanistan,Pakistan,AFG,Asia,PAK,371.0,0.00269541779
3,Afghanistan,Pakistan,AFG,Asia,PAK,371.0,0.00269541779
4,Afghanistan,Tajikistan,AFG,Asia,TJK,444.0,0.002252252252
...,...,...,...,...,...,...,...
820,Zambia,Zimbabwe,ZMB,Africa,ZWE,347.0,0.00288184438
821,Zimbabwe,Botswana,ZWE,Africa,BWA,932.0,0.001072961373
822,Zimbabwe,Mozambique,ZWE,Africa,MOZ,1012.0,0.0009881422925
823,Zimbabwe,South Africa,ZWE,Africa,ZAF,NaN,NaN


In [33]:
#dropping Proximity in order to add manually, then it can be added again with the recalculation
if "Proximity_1_over_dist" in prox_final.columns:
    prox_final = prox_final.drop(columns=["Proximity_1_over_dist"], axis=1)
prox_final

,src,trg,iso_code_src,region,iso_code_trg,kmdist
0,Afghanistan,China,AFG,Asia,CHN,4199.0
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,IRN,1618.0
2,Afghanistan,Pakistan,AFG,Asia,PAK,371.0
3,Afghanistan,Pakistan,AFG,Asia,PAK,371.0
4,Afghanistan,Tajikistan,AFG,Asia,TJK,444.0
...,...,...,...,...,...,...
820,Zambia,Zimbabwe,ZMB,Africa,ZWE,347.0
821,Zimbabwe,Botswana,ZWE,Africa,BWA,932.0
822,Zimbabwe,Mozambique,ZWE,Africa,MOZ,1012.0
823,Zimbabwe,South Africa,ZWE,Africa,ZAF,NaN


In [34]:
#checking Nan's from the proximity list
prox_nans = prox_final[prox_final.isna().any(axis=1)]
prox_nans

,src,trg,iso_code_src,region,iso_code_trg,kmdist
16,Algeria,Western Sahara,DZA,Africa,ESH,NaN
17,American Samoa,Niue,ASM,Oceania,NIU,NaN
18,American Samoa,Samoa,ASM,Oceania,WSM,NaN
19,American Samoa,Wallis and Futuna,ASM,Oceania,WLF,NaN
21,Angola,"Congo, Democratic Republic of the",AGO,Africa,COD,NaN
...,...,...,...,...,...,...
807,Western Sahara,Algeria,ESH,Africa,DZA,NaN
808,Western Sahara,Mauritania,ESH,Africa,MRT,NaN
809,Western Sahara,Morocco,ESH,Africa,MAR,NaN
814,Zambia,"Congo, Democratic Republic of the",ZMB,Africa,COD,NaN


This is particularly tricky because we have to get the index in order to replace but we know the index of prox_nans, not prox_final. We should find an optimal way of searching through prox_final. 



In [35]:
#Manually adding KMdist

city_pairs = [('Algeria', 'Western Sahara', 1865.0 ),
              ('American Samoa', 'Niue', 538.0 ),
              ('American Samoa', 'Samoa', 124.0 ),
              ('American Samoa', 'Wallis and Futuna', 600.0 ),
              ('Angola', 'Congo, Democratic Republic of the', 500.0 ),
              ('Western Sahara', 'Mauritania', 976.0 ),
              ('Western Sahara', 'Morocco', 1865.0 ),
              ('Zambia', 'Congo, Democratic Republic of the', 1880.0),
              ('Anguilla', 'Antigua and Barbuda', 177.0 ),
              ('Anguilla', 'Saint Kitts and Nevis', 107.0 ),
              ('Aruba', "Venezuela (Bolivarian Republic of)", 407.0 ),
              ('Australia', 'Norfolk Island', 1900.0 ),
              ('Bermuda', 'United States of America', 1326.0 ),
              ('Botswana', 'South Africa', 1260.0 ),
              ('Brazil', 'French Guiana', 2354.0 ),
              ('Burundi', 'Congo, Democratic Republic of the', 2852.0 ),
              ('Canada', 'Greenland', 2558.0 ),
              ("Cayman Islands", 'Cuba', 429.0 ),
              ("Virgin Islands (British)", 'Anguilla', 167.0 ),
              ("Virgin Islands (British)", 'Puerto Rico', 156.0 ),
              ('Wallis and Futuna', 'Samoa', 480.0 ),
              ('Zimbabwe', 'South Africa', 2186.0 ),
              ('Uganda', 'South Sudan', 514.0 ),
              ("Cayman Islands", 'Jamaica', 508.0 ),
              ('Central African Republic', 'Congo, Democratic Republic of the', 1031.0 ),
              ('Central African Republic', 'South Sudan', 1444.0 ),
              ('China', 'Hong Kong', 1973.0 ),
              ('China', 'Macao', 1990.0 ),
              ('Turks and Caicos Islands', 'Bahamas', 750.0 ),
              ('Tuvalu', 'Wallis and Futuna', 731.0 ),
              ('Uganda', 'Congo, Democratic Republic of the', 2946.0 ),
              ('Christmas Island', "Cocos (Keeling) Islands", 984.0 ),
              ('Christmas Island', 'Indonesia', 491.0 ),
              ("Cocos (Keeling) Islands", 'Indonesia', 500.0 ),
              ("Congo", 'Congo, Democratic Republic of the', 20.0 ),
              ("Comoros", 'Mayotte', 750.0 ),
              ("Suriname", 'French Guiana', 330.0 ),
              ("Swaziland", 'South Africa', 1485.0 ),
              ("Tanzania, United Republic of", 'Congo, Democratic Republic of the', 3564.0 ),
              ('Tonga', 'American Samoa', 1101.0 ),
              ('Tonga', 'Fiji', 743.0 ),
              ("Congo, Democratic Republic of the", 'Rwanda', 2788.0 ),
              ("Congo, Democratic Republic of the", 'South Sudan', 2705.0 ),
              ("Cook Islands", 'Niue', 1086.0 ),
              ("Dominica", 'Guadeloupe', 84.0 ),
              ("Spain", 'Gibraltar', 496.0 ),
              ("Sudan", 'South Sudan', 1200.0 ),
              ("Dominican Republic", 'Puerto Rico', 400.0 ),
              ("Eritrea", 'South Sudan', 1415.0 ),
              ("Ethiopia", 'South Sudan', 915.0 ),
              ("Falkland Islands (Malvinas)", 'Argentina', 1901.0 ),
              ("Faroe Islands", 'United Kingdom of Great Britain and Northern Ireland', 1235.0 ),
              ("Fiji", 'New Caledonia', 1332.0 ),
              ("Fiji", 'Tuvalu', 1073.0 ),
              ("Fiji", 'Wallis and Futuna', 792.0 ),
              ("French Polynesia", 'Cook Islands', 1144.0 ),
              ("Greenland", 'Iceland', 1429.0 ),
              ("Guadeloupe", 'Antigua and Barbuda', 125.0 ),
              ("Guam", 'Northern Mariana Islands', 220.0 ),
              ("Guam", 'Palau', 1291.0 ),
              ("Guernsey", 'France', 361.0 ),
              ("Guernsey", 'United Kingdom of Great Britain and Northern Ireland', 284.0 ),
              ("Iceland", 'Faroe Islands', 802.0 ),
              ("Isle of Man", 'Ireland', 1482.0 ),
              ("Isle of Man", 'United Kingdom of Great Britain and Northern Ireland', 415.0 ),
              ("Jersey", 'France', 327.0 ),
              ("Jersey", 'United Kingdom of Great Britain and Northern Ireland', 293.0 ),
              ("Kenya", 'South Sudan', 894.0 ),
              ("Kiribati", 'United States of America', 11623.0 ),
              ("Lesotho", 'South Africa', 999.0 ),
              ("Madagascar", 'Mayotte', 726.0 ),
              ("Martinique", 'Dominica', 85.0 ),
              ("Mauritius", 'Réunion', 228.0 ),
              ("Marshall Islands", 'Nauru', 985.0 ),
              ("Mayotte", 'Mozambique', 1975.0 ),
              ("Mozambique", 'South Africa', 1622.0 ),
              ("Namibia", 'South Africa', 1270.0 ),
              ("Nauru", 'Micronesia (Federated States of)', 1278.0 ),
              ("New Caledonia", 'Australia', 2216.0 ),
              ("New Caledonia", 'Nauru', 2415.0 ),
              ("New Caledonia", 'Vanuatu', 540.0 ),
              ("Niue", 'Tonga', 600.0 ),
              ("Norfolk Island", 'New Zealand', 1493.0 ),
              ("Saint Lucia", 'Martinique', 66.0 ),
              ("Saint Pierre and Miquelon", 'Canada', 1508.0 ),
              ("Solomon Islands", 'Nauru', 1254.0 ),
              ("Madagascar", 'Réunion', 851.0 ),
             ]

def add_km(df, src_c, trg_c, km_c, l):
    
    for e in l:
        #print(e)
        a = set(df.index[prox_final[src_c] == e[0]].tolist())
        b = set(df.index[prox_final[trg_c] == e[1]].tolist())
        set_ab = a.intersection(b)
        c = set(df.index[prox_final[src_c] == e[1]].tolist())
        d = set(df.index[prox_final[trg_c] == e[0]].tolist())
        set_cd = c.intersection(d)
        
        final_l = list(set_ab)+list(set_cd)

        for i in final_l:
            
            df.at[i,km_c] = e[2]
            
            #print('Distance between {} and {} is {} km'.format(df.iloc[i][src_c],df.iloc[i][trg_c],df.iloc[i][km_c]))
            
    return df


#FIRST, FIND CORRECT INDEX, THEN REPLACE
#prox_final.at[16, 'kmlisdist'] = 1865
prox_final = add_km(prox_final, 'src', 'trg', 'kmdist', city_pairs)

In [36]:
#checking Nan's from the proximity list
prox_nans = prox_final[prox_final.isna().any(axis=1)]
prox_nans

,src,trg,iso_code_src,region,iso_code_trg,kmdist


In [37]:
prox_final[prox_final['src']=="American Samoa"]

,src,trg,iso_code_src,region,iso_code_trg,kmdist
17,American Samoa,Niue,ASM,Oceania,NIU,538.0
18,American Samoa,Samoa,ASM,Oceania,WSM,124.0
19,American Samoa,Wallis and Futuna,ASM,Oceania,WLF,600.0


In [38]:
#DROPPING LOW FLIGHT COUNTRIES

drop_l = ["American Samoa", 
          "Christmas Island", 
          "Cocos (Keeling) Islands", 
          "Cook Islands", 
          "Falkland Islands (Malvinas)",
          "French Polynesia",
          "Lesotho",
          "Marshall Islands",
          "Nauru",
          "Niue",
          "Saint Vincent and the Grenadines",
          "Swaziland",
          "Tuvalu",
          "Wallis and Futuna"
         ]

low_flights = prox_final[prox_final["src"].isin(drop_l)].index
low_flights2 = prox_final[prox_final["trg"].isin(drop_l)].index
i_drop = low_flights.union(low_flights2)

prox_final = prox_final.drop(i_drop)
#df = df.drop(df[<some boolean condition>].index)                         
                         
#df_new[df_new['l_ext'].isin([31, 22, 30, 25, 64])]

In [39]:
prox_final[prox_final['src'] == "American Samoa"]

,src,trg,iso_code_src,region,iso_code_trg,kmdist


In [40]:
#verifying integrity of the dataset
nan_values = prox_final.isna()
nan_columns = nan_values.any()

columns_with_nan = prox_final.columns[nan_columns].tolist()

In [41]:
columns_with_nan

[]

In [42]:
#verifying that there is no empty kmdist
prox_final.loc[prox_final['kmdist'] == '0']

/home/jan/anaconda3/envs/thesis/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,src,trg,iso_code_src,region,iso_code_trg,kmdist


In [43]:
# REMOVING BI-DIRECTIONAL LOOPS

def one_source_no_addition(df,s,t,w):
    #df2 = df.copy(deep=True)
    i_protecc = []
    i_to_del = []

    for idx, row in df.iterrows():

        if ((df[t] == row[s]) & (df[s] == row[t])).any():
            #print("Flights from {} to {} = {}".format(row[s], row[t],row[w]))
            
            i_row = df.loc[((df[t] == row[s]) & (df[s] == row[t]))].index[0]
            #print(df2.loc[i_row, w])
            
            #print("Flights from {} to {} = {}".format(df.loc[i_row, s], df.loc[i_row, t], df.loc[i_row, w]))          
            
            i_protecc.append(idx)
            if i_row not in i_protecc:
                i_to_del.append(i_row)
            #print(i_to_del)
            
        else:
            pass
    
    if i_to_del != []:
        df = df.drop(i_to_del)
    return df

In [44]:
prox_final = one_source_no_addition(prox_final, "src", "trg", "kmdist")


In [45]:
prox_final["proximity"] = 1 / prox_final["kmdist"]
prox_final["log-prox"] = 1 / np.log(prox_final["kmdist"])
prox_final

,src,trg,iso_code_src,region,iso_code_trg,kmdist,proximity,log-prox
0,Afghanistan,China,AFG,Asia,CHN,4199.0,0.000238,0.119867
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,IRN,1618.0,0.000618,0.135337
2,Afghanistan,Pakistan,AFG,Asia,PAK,371.0,0.002695,0.169027
3,Afghanistan,Pakistan,AFG,Asia,PAK,371.0,0.002695,0.169027
4,Afghanistan,Tajikistan,AFG,Asia,TJK,444.0,0.002252,0.164047
...,...,...,...,...,...,...,...,...
799,Venezuela (Bolivarian Republic of),Grenada,VEN,Americas,GRD,625.0,0.001600,0.155334
804,Virgin Islands (British),Anguilla,VGB,Americas,AIA,167.0,0.005988,0.195389
805,Virgin Islands (British),Puerto Rico,VGB,Americas,PRI,156.0,0.006410,0.198025
819,Zambia,"Tanzania, United Republic of",ZMB,Africa,TZA,1543.0,0.000648,0.136212


In [46]:
prox_final["norm-log-prox"]=((prox_final["log-prox"]-prox_final["log-prox"].min())/(prox_final["log-prox"].max()-prox_final["log-prox"].min()))

In [47]:
prox_final.to_csv('./data/0_Golden_Standard/proximity_normalized_06.csv')